<h1><center>  ADM HOMEWORK 3 -  Places of the world</center>

<b>Name</b>: Sara Cammarota <span style="color:white">--------i</span><b>Matricola</b>: 1869943 <span style="color:white">--i</span><b>Email</b>: cammarota.1869943@studenti.uniroma1.it  <br>
<b>Name</b>: Erica Luciani <span style="color:white">-------------ii</span><b>Matricola</b>: 1868647 <span style="color:white">--i</span><b>Email</b>: luciani.1868647@studenti.uniroma1.it <br>
<b>Name</b>: Javier Martinez Velasco <span style="color:white">-i</span><b>Matricola</b>: 2058968 <span style="color:white">--i</span><b>Email</b>: martinezvelasco.2058968@studenti.uniroma1.it <br>
<b>Name</b>: Francesco Obiso <span style="color:white">---------</span><b>Matricola</b>: 1908398 <span style="color:white">--i</span><b>Email</b>: obiso.1908398@studenti.uniroma1.it 

In [1]:
#General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter
import collections
import warnings
import string
from string import punctuation
from itertools import islice
import math
import json
import time
import os
import re

#Webscrapping
from bs4 import BeautifulSoup
import requests as rq
from datetime import datetime
from tqdm import tqdm
import bs4

#NLP
!pip install Unidecode
import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
nltk.download('stopwords')
nltk.download('punkt')
!python -m spacy download en_core_web_sm
!pip install -U spacy
import spacy

#Questions 3&4
!pip install geopy
import geopy.distance

#  1. Data collection

Here we detail the procedure used for the data collection.

## 1.1. Get the list of places

In the first place we manually <b>obtain the 400 URLs of the web's menu pages.

In [2]:
string1, string2 = "https://www.atlasobscura.com/places?page=" , "&sort=likes_count"
menu_urls = [string1+str(i)+string2 for i in range(1,401)]

After parsing the 400 <i>menu URLs</i> <b>we obtain a .txt file with the 7.200 URLs as a final output.

In [3]:
#We create a new file
file = open("7200_urls.txt", "x")

#We add to the new file all the urls that we obtaing from the 400 menu URLs
for i in tqdm(range(len(menu_urls))):
    page_content = rq.get(menu_urls[i])
    soup_content = BeautifulSoup(page_content.text)
    list_18_urls = ['https://www.atlasobscura.com'+ x.get('href') for x in soup_content.find_all('a', {'class':'content-card-place'})]
    file = open("7200_urls.txt", "a")
    file.write('\n'.join(list_18_urls) + '\n')
    file.close()

file.close()

##  1.2. Crawl places

We <b>create a list that contains all the urls</b> by iterating over the previously created file with 7.200 urls.

In [4]:
file = open("7200_urls.txt", "r")
file_iterable =file.readlines()
all_urls_list = [file_iterable[i][:-1] for i in range(len(file_iterable)) if file_iterable[i] != "\n"]        
file.close()

Inside a parent directory (<i>ADM_HW3_FOLDERS</i>), <b>we create 400 folders, each one of them cointaining 18 files, and every one of these files, contains the HTML code of a specific URL </b>. In order to obtain this output, we iterated over the list created in the previous step, which contains the 7200 URLs.
Also, is worth mentioning that we used the method <i>requests.Session()</i> which helped us to reduce time by mantaining the "<i>requesting session</i>" open. We also needed to include time.sleep(1) to avoid being blocked from the webpage.

In [5]:
parent_dir = "ADM_HW3_FOLDERS"
session = rq.Session()
for i in tqdm(range(313, 401)):
    directory = f'folder_{i}'
    os.makedirs(os.path.join(parent_dir,directory))
    for url in all_urls_list[i*18:(i+1)*18]:
        result = session.get(url)
        html = BeautifulSoup(result.text)
        filename = url[36:]
        parent_dir2 = f"ADM_HW3_FOLDERS/folder_{i}"
        path = os.path.join(parent_dir2,filename)
        new_file = open(path, "x")
        new_file.writelines(str(html))
        new_file.close()
        time.sleep(1)

## 1.3 Parse downloaded pages

<h4>Our Parser Function</h4> <br>
This is the function that scraps the important information of a given URL.

In [6]:
def extract_single_place(file):

    result = open(file,"r")
    soup = BeautifulSoup(result)
    
    placeName = soup.find_all('h1', {'class':'DDPage__header-title'})[0].contents[0]
    
    placeDesc = [x.text for x in soup.find_all('div', {'id': 'place-body'})]
    
    placeShortDesc = soup.find_all('h3', {'class': 'DDPage__header-dek'})[0].contents[0].replace(u'\xa0', u'')
    
    numPeopleVisited = int(soup.find_all('div', {'class':'title-md item-action-count'})[0].contents[0])
    
    numPeopleWant = int(soup.find_all('div', {'class':'title-md item-action-count'})[1].contents[0])
    
    nearby = soup.find_all('div', {'class':'DDPageSiderailRecirc__item-title'})
    placeNearby = [nearby[i].contents[0] for i in range(len(nearby)) if nearby[i].contents[0] != "\n" ]
    
    address = soup.find_all('address', {'class':'DDPageSiderail__address'})[0]
    street = address.find_all('div')[0].contents[0].replace('\n', '').strip()
    city = address.find_all('div')[0].contents[2].replace('\n', '').strip()
    country = address.find_all('div')[0].contents[4].replace('\n', '').strip()
    placeAddress = street +" "+city +" "+ country
    
    try:
        placeAlt = float(soup.find_all('div', {'class':'DDPageSiderail__coordinates js-copy-coordinates'})[0].contents[2][1:7])
    except ValueError:
        placeAlt = ""
    
    try:
        placeLong = float(soup.find_all('div', {'class':'DDPageSiderail__coordinates js-copy-coordinates'})[0].contents[2][10:-2])
    except ValueError:
        placeLong = ""
    
    editors = soup.find_all('a', {'class':"DDPContributorsList__contributor"})
    placeEditors = [editors[i].contents[0] for i in range(len(editors)) if editors[i].contents[0] != "\n"  ]
    
    try:
        date = soup.find_all('div', {'class':"DDPContributor__name"})[0].contents[0]
        placePubDate = datetime.strptime(date, '%B %d, %Y')
    except IndexError:
        placePubDate = ""
    
    try:
        placeTags = soup.find_all('div', {'class':'DDPage__header-place-location'})[0].contents[0]
        if type(placeTags) == bs4.element.NavigableString:
            placeTags = str(placeTags)
        else:
            placeTags = placeTags.contents[0]
    except IndexError:
        placeTags = ""
    
    try:
        placeRelatedLists = ""
        find_num_lists = soup.find_all('div', {'class':"CardRecircSection__title"})
        RelatedLists = soup.find_all('h3', {'class':"Card__heading --content-card-v2-title js-title-content"})
        if find_num_lists[2].contents[0][11:12] != " ":
            placeRelatedLists = [list(RelatedLists[-i].contents[1]) for i in range(1,int(find_num_lists[2].contents[0][11:12])+1)]
            placeRelatedLists = [item for sublist in placeRelatedLists for item in sublist]   
    except IndexError:
        placeRelatedLists = ""
        
    try:
        placeRelatedPlaces = ""
        find_rel_places = soup.find_all('div', {'class':"CardRecircSection__title"})
        RelatedPlaces = soup.find_all('h3', {'class':"Card__heading --content-card-v2-title js-title-content"})
        c =  find_rel_places[2].contents[0][11:12] 
        if c != " ": #we needed a -1 after the -i, when c = number , and we needed to iterate from c until c+4
            placeRelatedPlaces = [list(RelatedPlaces[-i-1].contents[1]) for i in range(int(c), int(c)+4)]
            placeRelatedPlaces = [item for sublist in placeRelatedPlaces for item in sublist]
            if int(c) >= 6 :
                placeRelatedPlaces = [list(RelatedPlaces[-i].contents[1]) for i in range(int(c), int(c)+4)]
                placeRelatedPlaces = [item for sublist in placeRelatedPlaces for item in sublist]
        else: #this block needed to don't be indented, to reach cases where c = " "
            placeRelatedPlaces = [list(RelatedPlaces[-i].contents[1]) for i in range(1, 5)]
            placeRelatedPlaces = [item for sublist in placeRelatedPlaces for item in sublist]    
    except IndexError:
        placeRelatedPlaces = ""
        
    placeUrl = "https://www.atlasobscura.com/places/" + os.path.basename(file)
    result.close()
    return {'placeName': placeName, 
            'placeTags': placeTags,
            'placeDesc': placeDesc[0].replace(u'\xa0', u' ').replace('\n', '').strip(),
            'placeShortDesc':placeShortDesc,
            'numPeopleVisited':numPeopleVisited,
            'numPeopleWant': numPeopleWant,
            'placeNearby': " , ".join(placeNearby),
            'placeAddress': placeAddress,
            'placeAlt': placeAlt,
            'placeLong':placeLong,
            'placeEditors': " , ".join(placeEditors),
            'placePubDate': placePubDate,
            'placeRelatedLists': " , ".join(placeRelatedLists),
            'placeRelatedPlaces': " , ".join(placeRelatedPlaces),
            'placeUrl': placeUrl}

The function `file_iterator_parser()` creates a new tsv file with the desired content for every file in our structure of 400 folders and 18 files per folder (7.200 files): <br>

In [7]:
def file_iterator_parser(filenames):
    #iterate over each folder
    for i in tqdm(range(400)):
        #filenames we are interested in:
        files = filenames[i*18:(i+1)*18] #for each iteration we choose only the file names we are interested in
        #iterate over each file in that specific folder
        for j in range(18):
            #parse the file below
            parsed_content = extract_single_place(f"ADM_HW3_FOLDERS/folder_{i}/{files[j]}")
            #turn it into a dataframe
            dataframe_content = pd.DataFrame(parsed_content, index = [0])
            #create a new tsv file in the general tsv directory ADM_HW3_TSV.
            dataframe_content.to_csv(f"NEW_TSVS/{files[j]}.tsv", sep = "\t")

            
#New list that contains all the names of our files. Needed for the next step.
filenames = [i[36:]for i in all_urls_list.copy()]

file_iterator_parser(last_part_urls)

The function `concatenate_tsvs()` creates a unique Dataframe by concatenating all 7.200 files present in our main directory: <br>

In [11]:
def concatenate_tsvs(list_of_files):
    file1 = pd.read_csv(f"NEW_TSVS/{list_of_files[0]}", sep = "\t")
    final_data = pd.DataFrame(file1, index = [0])

    for i in range(1,7200):
        file = pd.read_csv(f"NEW_TSVS/{list_of_files[i]}", sep = "\t")
        dataframe_content = pd.DataFrame(file, index = [0])
        final_data = pd.concat([final_data,dataframe_content],ignore_index=True)
    
    df_places = final_data.drop('Unnamed: 0', axis=1)
    
    return df_places

#We create a list with all our.tsv files stored in our main directory.
directory = "NEW_TSVS"
all_tsv_files = os.listdir(directory)
all_tsv_files= [i for i in all_tsv_files if i !=".ipynb_checkpoints"]

df_places = concatenate_tsvs(all_tsv_files)
df_places.head(5)

,placeName,placeTags,placeDesc,placeShortDesc,numPeopleVisited,numPeopleWant,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeUrl
0,Swing at the End of the World,"Banos, Ecuador",Deep in the Ecuadorian wilderness is a seismic...,A swing to dangle you off the very edge.,188,1241,"Momia de Guano , Chimborazo , Iglesia de Balba...",Casa del Arbol Banos Ecuador,-1,-78.0,"penelopeelizabeth , theghostofchristmaspast666...",2014-11-03,50 Obscure and Amazing Places to Visit in 2017...,"Fantasy of Flight , Sloss Furnaces , Edison's ...",https://www.atlasobscura.com/places/swing-at-t...
1,Tinglewood Carvings,"Montevallo, Alabama","Montevallo, Alabama’s Orr Park is a lovely nat...",Orr Park's dead trees have been carved into a ...,247,925,Pettus Randall Miniature Museum of American Hi...,"277 Park Dr Montevallo, Alabama, 35115 United ...",33,-87.0,"ladypalutena , wendiraven , tinewatts , Collec...",2016-03-22,NaN,"Brea Redwood Grove , Crooked Slide Park , Doll...",https://www.atlasobscura.com/places/tinglewood...
2,Berlin Wall Urinal,"Las Vegas, Nevada","When the Berlin Wall came down in 1989, it was...",A piece of the iconic wall found an unlikely n...,349,799,"The First Telephone in Las Vegas , St. Valenti...","200 N Main St Las Vegas, Nevada United States",36,-115.0,"4n4l0gb0y , MyDoctorWinksTooMuch",2017-10-12,10 Bathrooms You Should Pee in Before You Die,"Heisenberg's Bathtub , Văcărești Natural Park ...",https://www.atlasobscura.com/places/berlin-wal...
3,Tintern Abbey,"Tintern, Wales",Tintern Abbey is an extremely well-preserved C...,This well-preserved Cistercian Abbey in a roma...,508,859,"St Briavels Castle , Lydney Camp , Puzzlewood","A466 Tintern, Wales United Kingdom",52,-3.0,"Thomas Poston , old1475 , knoflookperser , rug...",2016-07-10,NaN,"The Nashville Clayscape , Zemun Fortress , Vic...",https://www.atlasobscura.com/places/tintern-abbey
4,Mount Angel Abbey Museum,"Mount Angel, Oregon",The Mount Angel Abbey Museum is a very eclecti...,World's largest porcine hairball sits in a col...,268,572,"The Gordon House , Wooden Shoe Tulip Farm , T...","One Abbey Drive Mount Angel, Oregon, 97373 Uni...",45,-123.0,"fatpirate , darkdaybreak , Nicholas Jackson",2011-02-09,NaN,"Milwaukee Art Museum , St. George Icon Museum ...",https://www.atlasobscura.com/places/mount-ange...


In [12]:
df_places.isna().sum()

placeName                0
placeTags                1
placeDesc                0
placeShortDesc           0
numPeopleVisited         0
numPeopleWant            0
placeNearby              0
placeAddress             0
placeAlt                 0
placeLong                1
placeEditors             6
placePubDate             1
placeRelatedLists     4448
placeRelatedPlaces      29
placeUrl                 0
dtype: int64

## 2. Search Engine

Upload a reduced version of Dataframe.

In [13]:
reduced_df = pd.read_csv("data.places.tsv", sep = '\t', usecols = ['placeName', 'placeDesc', 'placeUrl'])

For stopwords, we merged spacy's and nltk stopwords lists'. Once intersected both sets, we turn them into a list and remove all apostrophes, as we will compare this list with the stemmed descriptions, which don't have apostrophes.

In [14]:
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = nlp.Defaults.stop_words
nltk_stopwords = set(nltk.corpus.stopwords.words('english'))
all_stop_words = nltk_stopwords.union(spacy_stopwords)
all_stop = list(all_stop_words)
all_words = [all_stop[i].replace("'","").replace("‘","").replace("’","") for i in range(len(all_stop_words))]

We created a regex pattern that substitutes every character that is not a word or a number into a space character. We believe this is a desirable behaviour because not only removes punctuation but it also allows us to treat words with "-" in between (i.e. <i>first-timer</i>) in a better way. The output of the cell below is a new column in the dataframe ('Stemmed_description') which includes the "preprocessed version" of all words in the placeDesc column.

In [15]:
def stemming(column = reduced_df['placeDesc'],stopwords = all_words):
    ps = PorterStemmer() 
    stemmed = []
    for description in reduced_df['placeDesc']: 
        description = unidecode.unidecode(description.lower()) #treat accents and uppercase
        description = re.sub('[^a-zA-Z0-9]', ' ', description).split() #remove punct and "in between "-"s"
        description = [word.strip() for word in description if word not in all_words] #remove stopwords
        stemmed_desc = [ps.stem(word) for word in description] #perform stemming
        description = ' '.join(stemmed_desc)
        stemmed.append(description)
    
    return stemmed

When performing some exploratory data analysis on the dataset we identified that 20 places shared their name with at least other place. This made important to use Document IDs in throughout the homework, in order to avoid errors.

In [16]:
#1. Preprocess all the text.
preprocessed = stemming(column = reduced_df['placeDesc'],stopwords = all_words)
#2. Add new column with preprocessed content to dataframe
reduced_df['Stemmed_description'] = preprocessed
#3. Add new column with Doc_id to dataframe. There are 20 placeNames that are repeated.
reduced_df = reduced_df.reset_index()
reduced_df = reduced_df.rename(columns={'index': 'ID'})
reduced_df['ID'] = 'Doc_' + reduced_df['ID'].astype(str)
reduced_df.head(2)

,ID,placeName,placeDesc,placeUrl,Stemmed_description
0,Doc_0,Swing at the End of the World,Deep in the Ecuadorian wilderness is a seismic...,https://www.atlasobscura.com/places/swing-at-t...,deep ecuadorian wilder seismic monitor station...
1,Doc_1,Tinglewood Carvings,"Montevallo, Alabama’s Orr Park is a lovely nat...",https://www.atlasobscura.com/places/tinglewood...,montevallo alabama orr park love natur area fi...


### 2.1. Conjunctive query

#### 2.1.1) Create your index!

In the next step we create a vocabulary out of all the stemmed words.

In [18]:
words = set()
vocab = {}
term_id = 0
#Create vocabulary with all unique words
for text in tqdm(reduced_df['Stemmed_description']):
    text = set(text.split()) #unique words in a place description
    words = words.union(text) #add to the main vocabulary
#Assign a term_id
for word in words:
    vocab[word] = term_id
    term_id += 1

#Show a partial view of the dictionary
dict(islice(vocab.items(), 0, 5)) #40,029 unique words

100%|██████████| 7200/7200 [00:11<00:00, 616.63it/s] 


{'exuberantli': 0, 'brute': 1, 'shinga': 2, 'huge': 3, 'barnstorm': 4}

In order to mantain the exact associaton between each word and its term_id, we exported the dictionary in a json file and load it every time we run the notebook.

In [19]:
#Load JSON file
with open('vocabulary.json') as json_file:
    vocabulary = json.load(json_file)
dict(islice(vocabulary.items(), 0, 5))

{'exuberantli': 0, 'brute': 1, 'shinga': 2, 'huge': 3, 'barnstorm': 4}

Next step includes arraging the first inverted index. The resulting index was exported in a json file as: <i>inverted_index_v1.json.

In [20]:
def inverted_index_v1(vocabulary,reduced_df):
    InvertedIndex = {i: set() for i in range(len(vocabulary))} #new dict with word_id as keys.
    counter = 0
    for desc in reduced_df['Stemmed_description']:
        desc = desc.split()
        for word in desc: #add to the dict: key:word_id & values: docs where appears.
            InvertedIndex[vocabulary[word]].add(reduced_df.ID[counter])
        counter +=1 
    for key in InvertedIndex.keys(): #Convert the values of the dict in a list.
        InvertedIndex[key] = list(InvertedIndex[key])
    return InvertedIndex


#We create the inverted index:
inverted_index_v1 = inverted_index_v1(vocabulary,reduced_df)
#To show a partial view of the inverted index:
dict(islice(inverted_index_v1.items(), 0, 2))

{0: ['Doc_3524'], 1: ['Doc_1273', 'Doc_5865', 'Doc_6922']}

In [22]:
#Load JSON file
with open('inverted_index_v1.json') as json_file:
    inverted_index_v1 = json.load(json_file)
inverted_index_v1 = {int(k):v for k,v in inverted_index_v1.items()}

#### 2.1.2) Execute the query

First of all, we define a function that given a query, it is preprocessed in the same exact way as the stemmed descriptions. The output is a list containing the query's stemmed words.

In [23]:
def q():
    query = input()
    ps = PorterStemmer()
    query = unidecode.unidecode(query.lower()) #treat accents and uppercase
    query = re.sub('[^a-zA-Z0-9]', ' ', query).split() #remove punct and "in between "-"s"
    query = [word.strip() for word in query if word not in all_words]
    query = [ps.stem(word) for word in query]
    
    return query

#An example of the function, introducing the input "american museum".
q()

AmêRicAn-MuseÙM


['american', 'museum']

Secondly, in order to narrow our search, given a query, we are interested in finding a list with <b>only the documents that contain all the words within the query.

In [24]:
def documents_containing_whole_query(query,vocabulary,inverted_index_v1):
    if not query:
        documents = ""
    else:    
        documents = set(inverted_index_v1[vocabulary[query[0]]])
        for word in query[1:]:
            documents = documents.intersection(set(inverted_index_v1[vocabulary[word]]))

    return documents

#An example to show the output of the function
query = ["american","museum","test"]
documents_containing_whole_query(query,vocabulary,inverted_index_v1)

{'Doc_1729',
 'Doc_1964',
 'Doc_3188',
 'Doc_3313',
 'Doc_4605',
 'Doc_5071',
 'Doc_5571',
 'Doc_5924',
 'Doc_6777'}

Lastly, we define a function that, given a set of documents, <b>shows the user a dataframe</b> with only the documents containing the query made.

In [25]:
def relevant_places(documents,reduced_df):
    temp_df = pd.DataFrame(columns = ['placeName','placeDesc','placeUrl'])
    for doc_id in documents:
        temp_df = pd.concat([temp_df, reduced_df[reduced_df.ID==doc_id]], ignore_index = True)
    
    return display(temp_df[['placeName', 'placeDesc', 'placeUrl']])

### This is our first Search Engine:

In [27]:
#1. Input a query
query = q()

#2. We find all documents that contain all words within the query
documents = documents_containing_whole_query(query,vocabulary,inverted_index_v1)

#Show the info of these documents in a "temporal" dataframe.
relevant_places(documents,reduced_df)

american museum


,placeName,placeDesc,placeUrl
0,David Adickes Studio,David Adickes is known for the large statues h...,https://www.atlasobscura.com/places/david-adic...
1,Booze History Museum,Fair warning to all non-drinkers: This isn’t t...,https://www.atlasobscura.com/places/booze-hist...
2,Gadsby's Tavern,"Built in 1785, Gadsby’s Tavern in Old Town Ale...",https://www.atlasobscura.com/places/gadsbys-ta...
3,Philbrook Museum of Art,With its rich collection of historic and conte...,https://www.atlasobscura.com/places/philbrook-...
4,Theodore Roosevelt Birthplace Museum,Behind an otherwise innocuous (if immaculately...,https://www.atlasobscura.com/places/theodore-r...
...,...,...,...
252,Mount Zion Cemetery's Underground Railroad She...,Tucked away in the corner of the Mount Zion Ce...,https://www.atlasobscura.com/places/mount-zion...
253,National Cryptologic Museum,Update as of October 2021: Currently closed fo...,https://www.atlasobscura.com/places/national-c...
254,The Liberty Bell Hiding Place,"In September of 1777, the British defeated Geo...",https://www.atlasobscura.com/places/liberty-be...
255,Deutsches Jagd und Fischereimuseum (German Mus...,With its rows of taxidermy animal heads and di...,https://www.atlasobscura.com/places/deutsches-...


### 2.2) Conjunctive query & Ranking score

Next step is obtaining the TFIDF scores. At first we used <b>TfidfVectorizer</b> function from sklearn, but when trying to manually calculate the scores obtained, we didn't manage to reach the same results. Therefore, we decided to calculate the scores manually, which will give more transparency to the calculations and will allow us to use the same treatment for the query's vectorization.

<b><center>TFIDF =  Term Frequency * Inverse Document Frequency

- TF (TERM FREQUENCY) = total nº of repetitions of term T in document D / total nº of words in D

a) To obtain <b>total nº of repetitions of each term in each document</b> we use the Counter function from the Collections library, and iterate over all rows within the preprocessed/stemmed placeDesc column (in list format: "Stemmed_list").<br>
b) To find <b>total length of words in each document</b> we sum the frequencies of each word within each Counter object created in the previous step. We store as key the document id, and as values the total nº of words in that document. <br>
c) Finally, to find <b>Term Frequency</b>, we divide the two calculations previously obtained above, and get as an output a dictionary that contains as <b>key</b> the id of the document, and as <b>values</b> all the words and their respective TF scores within that document.<br>

In [28]:
#a) Total number of repetitions of each term in each document
reduced_df["Stemmed_list"] = [i.split(" ") for i in reduced_df["Stemmed_description"]]
word_counts_per_doc = {}
for i in range(len(reduced_df)):
    word_counts_per_doc[reduced_df["ID"][i]] = Counter(reduced_df["Stemmed_list"][i]) 
#Partial show of the word Counter per document:
dict(islice(word_counts_per_doc.items(), 0,1))

In [29]:
#b) Total number of words in each document
length_docs = {}
for i in reduced_df.ID:
    length_docs[i] = sum(word_counts_per_doc[i].values())
#Partial show of the lenght per document:
dict(islice(length_docs.items(), 0,5))

{'Doc_0': 139, 'Doc_1': 128, 'Doc_2': 102, 'Doc_3': 99, 'Doc_4': 83}

In [30]:
#c) TF (Term Frequency) for each word in each document
TF = {}
for i in word_counts_per_doc:
    tf_row = {}
    for word in word_counts_per_doc[i].keys():
        tf_row[word] =  word_counts_per_doc[i][word] / length_docs[i]
    TF[i] =  tf_row  
#Partial show of Term Frequencies
dict(islice(TF.items(), 0,1))

- IDF (INVERSE DOC FREQUENCY) = log (nº of documents / nº of documents that contain term T)

a) In order to obtain IDF of each word, we first need to obtain the number of documents in which each term appears.<br>
b) We calculate for each term log(7200/nºdocs where it appears).

In [31]:
#a) Number of documents that contain each term
number_docs = {}
for word in vocabulary:
    counter = 0
    for i in range(len(reduced_df)):
        if word in word_counts_per_doc[reduced_df["ID"][i]].keys():
            counter+=1
    number_docs[word] = counter
#Partial show of the nº of documents in which each term appears
dict(islice(number_docs.items(), 0,5))

{'exuberantli': 1, 'brute': 3, 'shinga': 1, 'huge': 391, 'barnstorm': 3}

In [32]:
#b) Calculate IDF for every term
IDF = {}
for i in word_counts_per_doc:
    idf_row = {}
    for word in word_counts_per_doc[i].keys():
            idf_row[word] =  np.log(7200/number_docs[word])
    IDF[i] = idf_row   
#Partial show of the IDF score of each term
dict(islice(IDF.items(), 0,1))

- TFIDF = TF*IDF 

In this step <b>we calculate TFIDF scores for each term in each document</b>, and simultaneusly, <b>we create an inverted index for each document</b>, which has term id as key and as value, a tuple containing the name of the document and the calculated TFIDF score for that word in that particular document.

In [33]:
#TFIDF for each term in each doc 
TFIDFs = {}
for i in reduced_df["ID"]:
    tfidf_row = {}
    for word in word_counts_per_doc[i].keys():
        tfidf_row[vocabulary[word]] =  tuple((i , np.log(7200/number_docs[word])*(word_counts_per_doc[i][word] / length_docs[i])))
    TFIDFs[i] =  tfidf_row  

#Partial show of the TFIDF score of each term in each document
dict(islice(TFIDFs.items(), 0,1))

#### 2.2.1) Inverted index

When calculating TFIDF scores, we created a dictionary made of 7,200 dictionaries, one per document. Each one of them contains all the words appearing on that document as key, and a tuple made of "doc_id" and "TFIDF score" as a value. Therefore, to obtain our final inverted index, we just need to merge all these dictionaries into one. Also, is worth mentioning that, for every word id (keys), the tuples (values) will be sorted according to the document id.

In [35]:
inverted_index_v2 = defaultdict(list)
for d in TFIDFs.values(): # you can list as many input dicts as you want here
    for key, value in d.items():
        inverted_index_v2[key].append(value)

#Show a partial view of the Inverted Index. i.e: Term 20543, corresponding to word "turner"
#appears in 16 docuements, having a different TFIDF score in each one of them.
dict(islice(inverted_index_v2.items(), 305,306)) 

{20543: [('Doc_3', 0.0617095715430744),
  ('Doc_2863', 0.0048066464065809325),
  ('Doc_3193', 0.1026764299624263),
  ('Doc_3220', 0.032153934646128236),
  ('Doc_3292', 0.18101474319301825),
  ('Doc_3688', 0.02545519826151819),
  ('Doc_3712', 0.08853982004006326),
  ('Doc_4120', 0.028415105036113327),
  ('Doc_4186', 0.12218495165528731),
  ('Doc_4259', 0.1328097300600949),
  ('Doc_4687', 0.03916184347925875),
  ('Doc_4997', 0.07215646751296495),
  ('Doc_6225', 0.019394436770680524),
  ('Doc_6439', 0.05007579985872431),
  ('Doc_6644', 0.02443699033105746),
  ('Doc_6784', 0.03593675048684921)]}

In [49]:
#Load JSON file
with open('inverted_index_v2.json') as json_file:
    inverted_index_v2 = json.load(json_file)
inverted_index_v2 = {int(k):v for k,v in inverted_index_v2.items()}

#### 2.2.2) Execute the query

In order to compare the cosine similarity of the query with the documents, <b>we need to obtain a vector with TFIDF scores for all words within the query</b>. Previously we obtained IDF scores for all terms, but they were stored in a complex structure that was convenient for arranging the inverted index. However, for calculating the IDF of the query¡s words, will be easier to work with a dict with term_id as key as IDF as value: 

In [36]:
IDF_scores = {}
for key in vocabulary.keys():
    IDF_scores[vocabulary[key]] = np.log(7200/len(inverted_index_v1[vocabulary[key]]))
#Partial view of IDF scores for each of the 40.029 terms in the vocabulary.
dict(islice(IDF_scores.items(), 0, 2))

{0: 8.881836305004146, 1: 7.783224016336037}

The function below takes as input the query and returns a vector containing the TFIDF scores off all terms in that query

In [37]:
def query_vector(query,vocabulary,IDF_scores):
    TF_query = {}
    for word in query: #Calculate TF for each term of the query
        TF_query[vocabulary[word]] = query.count(word)/len(query) 
    TFIDF_query = {}
    for key in TF_query.keys(): #Calculate TF*IDF for each word of the query  
        TFIDF_query[key] = IDF_scores[key]*TF_query[key]
    return list(TFIDF_query.values())

#An example to show the output of the function
query = ["american","museum","test"]  
query_vector(query,vocabulary,IDF_scores) #Output: TFIDFs of "american","museum","test".

[0.6953768432768774, 0.5219842668314538, 1.2881854893964073]

The function below takes as input the query and the relevant N documents, and returns <b>N vectors containing the TFIDF scores off all terms in that query, for each of the N documents.

In [38]:
def documents_vectors(query,documents,vocabulary,inverted_index_v2):
    vector_words = {}
    for word in query: #iterate trough all query's words.
        vector_word = {}
        for i in inverted_index_v2[vocabulary[word]]:#find word in inverted index. 
            if i[0] in documents: #if the tuple contains a relevant document...
                vector_word[i[0]]=i[1] #save its TFIDF score
        vector_words[word] = vector_word #save vector containing TFIDFs of the ith query word
    #Merge all dicts into one
    vectors = defaultdict(list)
    for d in vector_words.values(): 
        for key, value in d.items():
            vectors[key].append(value)
    return vectors

#An example to show the output of the function
query = ["american","museum","test","big"]
documents = documents_containing_whole_query(query,vocabulary,inverted_index_v1)
documents_vectors(query,documents,vocabulary,inverted_index_v2)

defaultdict(list,
            {'Doc_5071': [0.007119899419217176,
              0.005344548807147992,
              0.013189612519417141,
              0.02024323714946561]})

Next step is to compute cosine similarity, for which we have decided to create a Dataframe that shows all relevant documents as rows, and all words in the query as columns, having as values their respective TFIDFS.

In [39]:
def similarity_dataframe(vectors,query,query_array):
    similarity_df = pd.DataFrame.from_dict(vectors, orient='index', dtype=None , columns = query)
    similarity_df["vector"] = similarity_df.values.tolist()
    similarity_df.columns = similarity_df.columns.get_level_values(0)
    similarity_df = similarity_df.reset_index()
    similarity_df = similarity_df.rename(columns={'index': 'ID'})
    similarity_df["Similarity"] = [np.dot(i,query_array )/(np.linalg.norm(i)*np.linalg.norm(query_array)) for i in similarity_df.iloc[: , -1].values]
    
    return similarity_df

#An example to show the output of the function
query = ["american","museum","test"]
documents = documents_containing_whole_query(query,vocabulary,inverted_index_v1)
query_array = query_vector(query,vocabulary,IDF_scores)
vectors = documents_vectors(query,documents,vocabulary,inverted_index_v2)
similarity_dataframe(vectors,query,query_array)

,ID,american,museum,test,vector,Similarity
0,Doc_1729,0.014191,0.031958,0.131447,"[0.014191364148507704, 0.031958220418252276, 0...",0.926595
1,Doc_1964,0.040639,0.030506,0.025095,"[0.04063890642527206, 0.030505574035604447, 0....",0.868637
2,Doc_3188,0.020254,0.045610,0.037520,"[0.020253694464375072, 0.045610275742554225, 0...",0.888582
3,Doc_3313,0.007670,0.011514,0.014208,"[0.007669597536142031, 0.011514358827164424, 0...",0.961887
4,Doc_4605,0.010809,0.048682,0.020024,"[0.010808966475806387, 0.04868247048169, 0.020...",0.703107
5,Doc_5071,0.007120,0.005345,0.013190,"[0.007119899419217176, 0.005344548807147992, 0...",1.000000
6,Doc_5571,0.009190,0.027594,0.051073,"[0.00919000233405565, 0.027593881947037213, 0....",0.947907
7,Doc_5924,0.013725,0.051512,0.025425,"[0.013724542959412056, 0.05151160527941979, 0....",0.753709
8,Doc_6777,0.015685,0.023548,0.029057,"[0.015685191953613776, 0.023548162413449047, 0...",0.961887


Lastly, we create a function that merges the original dataframe with the one that contains the similarity, we sort by similarity and show the <b>top 10 results.

In [40]:
def query_result(reduced_df,similarity_df):
    k_most_similar_df = pd.merge(reduced_df[["ID","placeName","placeDesc","placeUrl"]], similarity_df[["ID","Similarity"]], on= 'ID')
    k_most_similar_df = k_most_similar_df[["placeName","placeDesc","placeUrl","Similarity"]]
    k_most_similar_df = k_most_similar_df.sort_values(by = 'Similarity', ascending=False).head(10)
    k_most_similar_df = k_most_similar_df.reset_index()
    k_most_similar_df = k_most_similar_df.drop("index",axis=1)
    return k_most_similar_df

### This is the new search engine:

In [41]:
#1. Input a query
query = q()
#2.Find documents that contain all words within the query
documents = documents_containing_whole_query(query,vocabulary,inverted_index_v1)
#3.Create a vector with TFIDF scores of the words in the query
query_array = query_vector(query,vocabulary,IDF_scores)
#4. Create one vector per each unique word in the query, with TFIDF scores in each document
vectors = documents_vectors(query,documents,vocabulary,inverted_index_v2)
#5. Create a Dataframe that computes the
similarity_df = similarity_dataframe(vectors,query,query_array)
#6. Merge it by ID with the original dataframe and sort by similarity
k_most_similar_df = query_result(reduced_df,similarity_df)
if not k_most_similar_df.empty:
    display(k_most_similar_df)
else:
    print("Sorry, no results for this search. Try again!")

castle england medieval


,placeName,placeDesc,placeUrl,Similarity
0,Hammond Castle Museum,John Hayes Hammond Jr. was a resourceful man. ...,https://www.atlasobscura.com/places/hammond-ca...,1.000000
1,The Caves of Nottingham,Underneath the modern city of Nottingham is a ...,https://www.atlasobscura.com/places/nottingham...,0.984682
2,Bodiam Castle,This picture-perfect moated medieval castle wa...,https://www.atlasobscura.com/places/bodiam-castle,0.903656
3,Strawberry Hill,This “little gothic castle” on the Thames spar...,https://www.atlasobscura.com/places/strawberry...,0.881948
4,Tintagel Castle,Tintagel Castle stands on windswept cliffs in ...,https://www.atlasobscura.com/places/tintagel-c...,0.867935
5,Graylyn Estate,Visitors to North Carolina may not expect to c...,https://www.atlasobscura.com/places/graylyn,0.845114
6,Alnwick Castle,Alnwick Castle is not only one of the largest ...,https://www.atlasobscura.com/places/alnwick-ca...,0.757747
7,Ye Olde Trip to Jerusalem,Built on a series of caves under Nottingham Ca...,https://www.atlasobscura.com/places/ye-olde-tr...,0.738358
8,Restormel Castle,There are many wonderful things about this med...,https://www.atlasobscura.com/places/restormel-...,0.669557
9,Nuremberg Castle,Nuremberg and the castle it was centered aroun...,https://www.atlasobscura.com/places/nuremberg-...,0.669557


### 3. Define a new score!

We chose to compute two new metrics. The first is based on the popularity of each place, and it's computed as the mean between the number of people who visited the place and the number of people who want to visit the place. The second score is the distance between the user's coordinates and the place coordinates. 

The user can choose to be shown the most popular, less crowded or closest places. We also ask the users how many places they want to see. 

In [44]:
# Add new column with Doc_id to dataframe.
df_places = df_places.reset_index()
df_places = df_places.rename(columns={'index': 'ID'})
df_places['ID'] = 'Doc_' + df_places['ID'].astype(str)
df_places.head(2)

,ID,placeName,placeTags,placeDesc,placeShortDesc,numPeopleVisited,numPeopleWant,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeUrl
0,Doc_0,Swing at the End of the World,"Banos, Ecuador",Deep in the Ecuadorian wilderness is a seismic...,A swing to dangle you off the very edge.,188,1241,"Momia de Guano , Chimborazo , Iglesia de Balba...",Casa del Arbol Banos Ecuador,-1,-78.0,"penelopeelizabeth , theghostofchristmaspast666...",2014-11-03,50 Obscure and Amazing Places to Visit in 2017...,"Fantasy of Flight , Sloss Furnaces , Edison's ...",https://www.atlasobscura.com/places/swing-at-t...
1,Doc_1,Tinglewood Carvings,"Montevallo, Alabama","Montevallo, Alabama’s Orr Park is a lovely nat...",Orr Park's dead trees have been carved into a ...,247,925,Pettus Randall Miniature Museum of American Hi...,"277 Park Dr Montevallo, Alabama, 35115 United ...",33,-87.0,"ladypalutena , wendiraven , tinewatts , Collec...",2016-03-22,NaN,"Brea Redwood Grove , Crooked Slide Park , Doll...",https://www.atlasobscura.com/places/tinglewood...


In [23]:
warnings.filterwarnings("ignore")

#1. Input a query
query = q()
#2. Find documents that contain all words within the query
places = documents_containing_whole_query(query,vocabulary,inverted_index_v1)
#3. Ask the user how many places (k) to be shown
print('How many places you would to like to see?')
k = int(input())
#4. Ask the user the searching criteria
print('\n Press: \n 1. Most popular places \n 2. Less visited places \n 3. Places near to you')
choice = int(input())

# Reducing the whole df taking only the rows of the places that contain all the words in the query
reduced_df2 = df_places[df_places['ID'].isin(places)]


# if users want to see the most popular places or the less popular places
if choice == 1 or choice == 2:
    # adding a col with the new score 'popularity' computed as the mean of numPeopleVisited and numPeopleWant
    reduced_df2['placeScore'] = (reduced_df2['numPeopleVisited'] + reduced_df2['numPeopleWant']) / 2
    # sorting the dataframe with respect to the new score
    reduced_df2 = reduced_df2.sort_values(by = 'placeScore')
    # results
    if choice == 1:
        output = reduced_df2[['placeName', 'placeDesc', 'placeUrl', 'placeScore']].tail(k)
    elif choice == 2:
        output = reduced_df2[['placeName', 'placeDesc', 'placeUrl', 'placeScore']].head(k) 
# if users want to see the places near to them
if choice == 3:
    # asking the users their position in coordinates
    print('Insert your coordinates (latitude longitude): \n')
    alt, long = map(int, input().split())
    #computing the distance between the position of the user and the position of each place, it will bw the new score
    latitudes = list(reduced_df2['placeAlt'])
    longitudes = list(reduced_df2['placeLong'])
    scores = [geopy.distance.geodesic((alt, long), (latitudes[i], longitudes[i])).km for i in range(len(reduced_df2))]
    # adding a col with the new score 'distance' and sorting the dataframe
    reduced_df2['placeScore'] = scores
    # result
    output = reduced_df2.sort_values(by = 'placeScore', ascending = True).head(k)

#print
output

museum
How many places you would to like to see?
5

 Press: 
 1. Most popular places 
 2. Less visited places 
 3. Places near to you
3
Insert your coordinates (latitude longitude): 

41 12


,ID,placeName,placeTags,placeDesc,placeShortDesc,numPeopleVisited,numPeopleWant,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeUrl,placeScore
3881,Doc_3881,Villa of Tiberius,"Sperlonga, Italy",Fascinating ruins and a huge grotto peer up at...,The Roman Emperor's lost summer home and grott...,73,579,"Via Flacca Ruins , Appia Antica in Itri , Jewi...",Via Flacca Km Sperlonga Italy,41,13.0,"mulishwhim , voyager16",2017-10-13,NaN,"Ambleside Roman Fort , Chedworth Roman Villa ,...",https://www.atlasobscura.com/places/villa-and-...,84.134725
5887,Doc_5887,Vicus Caprarius,"Rome, Italy",The Trevi Fountain is one of Rome’s most notab...,"A forgotten ""city of water"" underneath the fam...",98,613,Papal Hearts at Santi Vincenzo e Anastasio a T...,"25 Vicolo del Puttarello Rome, 00187 Italy",42,12.0,"Fred Cherrygarden , napostrophe",2020-09-16,NaN,"Casa Romuli , Resti del Ponte Romano (Roman Br...",https://www.atlasobscura.com/places/vicus-capr...,111.063588
3271,Doc_3271,The Museum of Roman Ships at Fiumicino,"Rome, Italy",An accidental discovery delivered this unique ...,Roman-era ships unearthed...,73,425,Secret Compartment in Leonardo da Vinci Statue...,15 Via Alessandro Guidoni Fiumicino Airport Ro...,42,12.0,"Rebekah Otto , mbison , ricorico913 , diegoton...",2010-02-08,NaN,"Springs Park , Jekyll Island Amphitheater , F...",https://www.atlasobscura.com/places/museum-rom...,111.063588
2159,Doc_2159,Mamertine Prison,"Rome, Italy",While jails and prisons were not a common feat...,This ancient Roman prison is decorated with an...,260,685,"Lacus Curtius , The Globe of the Vatican Obeli...","Via Clivo Argentario 1 Rome, 00186",42,12.0,"alleywaykid , Saradactyl",2014-12-12,"The World Turned Upside Down , 10 Places to Ex...","Clifton Cliff Jail , Dragsholm Castle , Lincol...",https://www.atlasobscura.com/places/mamertine-...,111.063588
3741,Doc_3741,Palazzo Naiadi Roman Baths,"Rome, Italy",Amid the construction of the luxury Palazzo Na...,Ancient ruins hidden underneath a modern luxur...,329,438,"Monument to the Fallen of Dogali , Meridian Li...","Piazza della Repubblica, 47 Rome Italy",42,12.0,"dombrunaccioni , unitof , Collector of Experie...",2018-09-12,NaN,"Parco Archeologico di Velia , Pompeii Thermopo...",https://www.atlasobscura.com/places/palazzo-na...,111.063588


### 4. Visualizing the most relevant places

We plot the map with the most relevant places, centered on the posistion of the user. 

In [24]:
import plotly.express as px

fig = px.scatter_mapbox(output, lat = "placeAlt", lon = "placeLong", center = {"lat": alt, "lon": long},  width = 1000,  
    height = 600, color="numPeopleVisited", size="numPeopleVisited", zoom=0.5, hover_data = ["placeName", "placeAddress", "numPeopleVisited"])
fig.update_layout(
    title={
        'text': "Maps visualization of the most relevant place",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_layout(mapbox_style="carto-positron") 
fig.show()

# 7. Theoretical Question

#### 1. Try solving the problem mentioned above using three different sorting algorithms (do not use any MapReduce algorithm). (Note: Built-in Python functions (like .mean, .sort, etc.) are not allowed to be used. You must implement the algorithms from scratch).

In [1]:
def mean(marks): #Given a list, the functions returns the mean of its components...
    s = 0
    for mark in marks:
        s += mark
    return round(s/len(marks), 2) #rounded to the second decimal 

In [2]:
with open('ApplicantsInfo.txt') as f: #opens the file 
    line0 = f.readline()              #reads the first line of the file to get n and m 
    line0 = line0.split()
    n = line0[0]
    m = line0[1]
    l = []
    Lines = f.readlines()             #reads the other lines of the file
    for line in Lines:                #for each applicant we compute the mean and store his name and mean in a sublist 
        line = line.split()
        marks = [int(i) for i in line[2:]]
        l.append([line[0] + ' ' + line[1], mean(marks)])
    print(l)

##### Insertion Sort

In [90]:
def swap(x, y): #Given x and y the function switches x and y
    temp0 = y[0]
    temp1 = y[1]
    y[0] = x[0]
    y[1] = x[1]
    x[0] = temp0
    x[1] = temp1
    return 

def insertionsort(l):
    # we loop over the components of the list
    for i in range(1, len(l)):
        # we check whether the current element is greater or equal to the previous ones
        while ((i > 0) and (i-1 >= 0) and (l[i][1] >= l[i-1][1])):
                # if it's greater we switch them because we want to sort in descending order with respect to the means
                if l[i][1] > l[i-1][1]:
                    swap(l[i], l[i-1])
                    i -= 1
                    continue
                # if the means are equal we check what name comes first in alphabetical order
                elif l[i][1] == l[i-1][1]:
                    if l[i][0] != l[i-1][0]:
                        # and if the current element comes after we switch them
                        if l[i-1][0] > l[i][0]:
                            swap(l[i], l[i-1])  
                    i -= 1
    return l

sortedI = insertionsort(l)

In [91]:
file = open("RankingList.txt", "w")
write = [f'{elem[0]} {elem[1]} \n' for elem in sortedI]
file.writelines(write)
file.close()

##### Merge Sort

In [92]:
def merge(l, first, middle, last):
    i = first - 1
    j = middle 
    temp = []     #creating a temporary array
    
    while ((i < middle) and (j < last)):      #iterating over l. 
        
        #sorting by grade mean
        if l[i][1] > l[j][1]:             
            temp.append(l[i])               
            i += 1
                                           
        elif l[i][1] < l[j][1]:
            temp.append(l[j])
            j += 1
            
       #if the grade mean is the same, sort by alphabetical order     
        elif l[i][0] <= l[j][0]:
            temp.append(l[i])
            i += 1
                                            
        elif l[i][0] > l[j][0]:
            temp.append(l[j])
            j += 1
        
    while i < middle:       #while the first temporary array is incomplete:
        temp.append(l[i])
        i += 1

    while j < last:         #while the secondary array is incomplete:
        temp.append(l[j])
        j += 1
    
    for i in range(first-1, last):   #copy the temporary array on l 
        l[i] = temp[i-first+1]
    
    return l
        
        
def mergesort(l, first, last):
    if len(l) < 2:
        return l
    if first < last:
        middle = (first + last)//2
        mergesort(l, first, middle)
        mergesort(l, middle + 1, last)
        l = merge(l, first, middle, last)
    return l

sortedM = mergesort(l, 1, len(l))

In [93]:
file = open("RankingListMerge.txt", "w")
write = [f'{elem[0]} {elem[1]} \n' for elem in sortedM]
file.writelines(write)
file.close()

##### Quick Sort

In [3]:
def quicksort(l, first, last):
    if len(l) < 2:
        return l
    
    # we choose the last element of our list of lists as pivot 
    pivot = l[last]
    
    # we inizialize two empty lists where we will sort the elements smaller than pivot (left) and those greater (right)
    left = []
    right = []
    
    # we divide the elements of the list between the two lists
    for i in range(len(l)):
        # if the mean of l[i] is greater than the mean of pivot
        if l[i][1] > pivot[1]:
            right.append(l[i])
            
        # if l[i] and the pivot have the same mean, but the name of l[i] comes before the name of the pivot 
        elif l[i][1] == pivot[1] and l[i][0] < pivot[0]: 
            right.append(l[i])
            
        # if l[i] and the pivot have the same mean, but the name of l[i] comes after the name of the pivot 
        elif l[i][1] == pivot[1] and l[i][0] > pivot[0]:
            left.append(l[i])
            
        # if the mean of l[i] is greater than the mean of pivot
        elif l[i][1] < pivot[1]: 
            left.append(l[i])
            
    # we recursively apply the quicksort to the new sublists left and right
    sortedLeft = quicksort(left, 0, len(left)-1)
    sortedRight = quicksort(right, 0, len(right)-1)
    
    # and now we join the final sorted list
    l = sortedRight + [l[last]] + sortedLeft
    return l

In [4]:
sortedQ = quicksort(l, 0, len(l)-1)

In [5]:
file = open("RankingListQuick2.txt", "w")
write = [f'{elem[0]} {elem[1]} \n' for elem in sortedQ]
file.writelines(write)
file.close()

#### 2. What is the time complexity of each algorithm you have used?

##### Insertion Sort

Given a list of lists with lenght $n$, each value is compared to the adjacent value to its left and if it's smaller than this value we switch them (or, in our case, if it's greater). 
The best case occurs when the number of switches is minimum, hence when the list is already sorted in ascending order (or, in our case, in descending order). if so, the cost is linear: $T(n) = O(n)$ , because we iterate over the array just once.
The worst case occurs when the number of switches is maximum, hence when the list is sorted in in descending order (or, in our case, in ascending order). In this case the cost is not linear: $T(n) = O(n^2)$.

##### Merge Sort

Given a list l of lenght n, we divide the list in two sublists of lenght $\frac{n}{2}$ and then mergesort calls itself on the two sublists. The cost of the merge function is $\theta(n)$. Hence the needed time is: $$T(n)= 2T(n/2) + \theta(n).$$ By applying the principal theorem, we obtain that the cost is $T(n) = O(n logn).$

##### Quick Sort

Let's compute the cost of this algorithm. Given a list of lists l -let's call its lenght $n$-, we divide the list in two sublist left and right, with lentghs $k$ and $n-k$ respectively, and then we call the function quicksort again on the sublists left and right. The cost of the division is $\theta(n)$ since we have to inspect the whole list on which we call the quicksort. Hence we can write the time taken by the quicksort as follows: $$T(n) = T(k) + T(n-k) + \theta(n)$$
Now in order to get a more precise esteem of the required time, let's consider the worst, the best and the average cases:
- the worst case scenario occurs when the choosen pivot is always the greatest or the smallest mean in the list, hence when $k=1$ or $k=n-1$: $$ T(n) = T(n-1) + T(0) +\theta(n) = T(n-1) + \theta(n) \Rightarrow T(n) = O(n^2).$$
- the best case occurs when the choosen pivot is always the greatest the middle element, hence when $k=\frac{n}{2}$: $$ T(n) = 2T(\frac{n}{2})+\theta(n)  \Rightarrow T(n) = O(n logn)$$

#### 3. Evaluate the time taken for each of your implementations to answer the query stored in the ApplicantsInfo.txt file and visualize them.

##### Insertion Sort

In [99]:
st = time.time()
insertionsort(l)
et = time.time()
ins_execution_time = (et-st) / 60
print('Execution time:', ins_execution_time, 'minutes')

Execution time: 0.5098026990890503 minutes


##### Merge Sort

In [100]:
st = time.time()
mergesort(l, 1, len(l))
et = time.time()
merge_execution_time = (et-st) / 60
print('Execution time:', merge_execution_time, 'minutes')

Execution time: 0.007161434491475423 minutes


##### Quick Sort

In [10]:
st = time.time()
quicksort(l, 0, len(l)-1)
et = time.time()
quick_execution_time = (et-st) / 60
print('Execution time:', quick_execution_time, 'minutes')

Execution time: 0.01043097972869873 minutes


#### 4. What is the most optimal algorithm, in your opinion, and why?

In general there isn't an algorithm better than the others, it depends on what we are interested in, namely on what we consider optimal.
Since we are working with a large amount of data, we could consider the most optimal algorithm as the fastest one. If we consider the execution time, of course **the Insertion sort isn't really efficient**, since we want to sort 50.000 elements and in the worst case the Insertion Sort costs $O(n^2) = O(50.000^2)$.  
This also happens for the Quick Sort, but for this algorithm rarely the worst case occurs. So, in practice, **the most efficient one is Quick sort**, since, unlike the Merge Sort, it doesn't allocate new memory at each iteration.